In [3]:
import sys
import os
# add parent directory (where "natsume" lives) to sys.path so I can import natsume
sys.path.append(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME")
import natsume

import ttv_curvefit.curvefit as ttvfit  # custom-written
import ttvfast
from ttvfast import models

from tqdm import tqdm
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.constants import M_earth
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# Import CSV for TTVFast simulations
df = pd.read_csv(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME\validation\1st_order_TTVFast\lithwick_TTVFast_params.csv")
df.head()

,e1,w1,e2,w2,Delta,j,P1,P2,m1,m2,mu1,mu2,validity1,validity2,valid
0,0.217826,125.098870,0.011114,250.839745,0.008372,5,7,8.823254,34.522853,7.794373,0.000104,0.000023,13.744474,3.103149,False
1,0.043071,109.337070,0.042463,146.170882,-0.046252,2,7,13.352477,14.587718,1.592966,0.000044,0.000005,10380.979802,1133.593878,True
2,0.013426,149.951647,0.018693,68.549390,-0.032123,2,7,13.550281,2.894622,51.337945,0.000009,0.000154,357.999885,6349.353063,True
3,0.168492,198.228783,0.048897,10.946683,-0.030239,2,7,13.576657,103.900720,13.887060,0.000312,0.000042,130.110252,17.390148,False
4,0.012973,158.697509,0.027805,112.138555,-0.007286,4,7,9.265328,23.031962,1.107622,0.000069,0.000003,573.937740,27.601051,False


In [4]:
# Copy dataframe for fitting results
df_fit = df.copy()
df_fit_cols = ["A1", "B1", "V1", "Pttv1", "phase1", "std1", "res1",
               "A2", "B2", "V2", "Pttv2", "phase2", "std2", "res2",
               "R2_1", "R2_2"]
df_fit[df_fit_cols] = np.nan
df_fit.head()

,e1,w1,e2,w2,Delta,j,P1,P2,m1,m2,...,res1,A2,B2,V2,Pttv2,phase2,std2,res2,R2_1,R2_2
0,0.217826,125.098870,0.011114,250.839745,0.008372,5,7,8.823254,34.522853,7.794373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.043071,109.337070,0.042463,146.170882,-0.046252,2,7,13.352477,14.587718,1.592966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.013426,149.951647,0.018693,68.549390,-0.032123,2,7,13.550281,2.894622,51.337945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.168492,198.228783,0.048897,10.946683,-0.030239,2,7,13.576657,103.900720,13.887060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.012973,158.697509,0.027805,112.138555,-0.007286,4,7,9.265328,23.031962,1.107622,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# For storing TTVs
df_ttv_cols = ["t0", "ttv0", "t1", "ttv1"]
df_ttv = pd.DataFrame(index=range(len(df)), columns=df_ttv_cols, dtype=object)
df_ttv.head()

,t0,ttv0,t1,ttv1
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [6]:
def ttv_model(t, A, B, V, Pttv, phase):
    return A + B*t + V * np.sin(2*np.pi/Pttv * t + phase)

In [7]:
# Create TTV simulations and extract Amplitude and Period
gravity = 0.000295994511 # AU^3/day^2/M_sun
stellar_mass = 1.0       # M_sun

for i, row in tqdm(df.iterrows(), total=len(df)):
    # If invalid scenario, skip
    if row['valid'] == False:
        continue
    
    # Initialize variables
    P1 = row['P1']; P2 = row['P2']
    j = row['j']
    PTTV = 1/np.abs(j/P2 - (j-1)/P1)
    
    e1 = row['e1']; w1 = row['w1']
    e2 = row['e2']; w2 = row['w2']
    
    mu1 = row['mu1']; mu2 = row['mu2']
    
    # Create TTVFast planets
    planet1 = models.Planet(
        mass=mu1,            # M_sun
        period=P1,           # days
        eccentricity=e1,
        inclination=90,      # degrees
        longnode=0,          # degrees
        argument=90+w1,      # degrees
        mean_anomaly=0       # degrees
    )
    planet2 = models.Planet(
        mass=mu2,
        period=P2,
        eccentricity=e2,
        inclination=90,
        longnode=0,
        argument=90+w2,
        mean_anomaly=0,
    )
    planets = [planet1, planet2]
    
    begin_time = -P2 * 200
    Time = begin_time   # days
    dt = P1 / 100       # days
    Total = max(250, PTTV * 2.5)  # days
    
    results = ttvfast.ttvfast(planets, stellar_mass, Time, dt, Total,
                              input_flag=1)

    # Create TTV signal from TTVFast Output
    out = pd.DataFrame(results['positions']).transpose()
    out.columns = ['planet', 'epoch', 'times', 'rsky', 'vsky']
    
    table0 = out[(out.planet == 0) & (out.times > 0)]
    n0 = table0.epoch
    tn0 = table0.times

    table1 = out[(out.planet == 1) & (out.times > 0)]
    n1 = table1.epoch
    tn1 = table1.times

    # If few TTVFast output, skip
    len_threshold = 10
    if (len(n0) <= len_threshold) or (len(n1) <= len_threshold):
        continue
        
    time0, ttv0, ttv0_err = ttvfit.return_ttv(epochs=n0, midtransits=tn0)
    time1, ttv1, ttv1_err = ttvfit.return_ttv(epochs=n1, midtransits=tn1)
    
    # Make sure ttv0 and ttv1 actually are centered around O-C = 0
    ttv0 = ttv0 - (np.max(ttv0) + np.min(ttv0)) / 2
    ttv1 = ttv1 - (np.max(ttv1) + np.min(ttv1)) / 2
    
    # Get sinusoidal TTV model parameters
    popt_ttv0, perr_ttv0, r2_0 = ttvfit.characterize_ttv(time=time0, ttv=ttv0, ttv_err=ttv0_err,
                                                         expected_period=PTTV, method='lmfit')
    popt_ttv1, perr_ttv1, r2_1 = ttvfit.characterize_ttv(time=time1, ttv=ttv1, ttv_err=ttv1_err,
                                                         expected_period=PTTV, method='lmfit')
    std0, std1 = np.std(ttv0), np.std(ttv1)
    res0, res1 = np.std(ttv0 - ttv_model(time0, *popt_ttv0)), np.std(ttv1 - ttv_model(time1, *popt_ttv1)) # Residual dispersion

    # Write results to df_fit and df_ttv
    df_ttv.loc[i] = {
        "t0": time0,
        "ttv0": ttv0,
        "t1": time1,
        "ttv1": ttv1
    }
    df_fit.loc[i, df_fit_cols]  = [*popt_ttv0, std0, res0, *popt_ttv1, std1, res1, r2_0, r2_1]

100%|████████████████████████████████████████████████████████████████████████████| 50000/50000 [17:44<00:00, 46.96it/s]


In [8]:
# If output fit has no values, convert valid from TRUE to FALSE
output_has_nan = df_fit[df_fit_cols].isna().any(axis=1)
df_fit.loc[output_has_nan, 'valid'] = False

# If output has PTTV wildly different from each other, convert valid from TRUE to FALSE
Pttv_ratio_threshold = 1.01
Pttv_not_similar = (df_fit['Pttv1'] / df_fit['Pttv2'] > Pttv_ratio_threshold) | (df_fit['Pttv2'] / df_fit['Pttv1'] > Pttv_ratio_threshold)
df_fit.loc[Pttv_not_similar, 'valid'] = False

In [9]:
# Export dataframes
df_ttv.to_csv("lithwick_TTVFast_outputs.csv", index=False)
df_fit.to_csv("lithwick_TTVFast_fits.csv", index=False)